<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/computer_vision/object_detection/object_detection_api_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Пример поиска и распознавания объектов на изображении**

Используется TensorFlow Object Detection API. Инструкции по установке:

Linux.
Windows.
Видео на YouTube с демонстрацией распознавания объектов.


**Imports and Setup of Object Detection**

In [ ]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

**Сам код**

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
%matplotlib inline

# **Загружаем модель TensorFlow**
Модель можно загрузить из Tensorflow detection model zoo. Из каталога с моделью нужен всего один файл - frozen_inference_graph.pb. Это "замороженная" предварительно обученная сеть, которую можно использовать для распознавания (inference).

Есть два типа моделей, отличаются выходными данными, указано в слобце Outputs в таблице:

    Boxes - модель выдает прямоугольник, внутри которого находится найденный объект
    Masks - модель выдает маску пикселей, которые соответствуют объекту

In [ ]:
# Тип выходных данных модели - Boxes
model_file_name = 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28/frozen_inference_graph.pb'
# Тип выходных данных модели - Masks
# model_file_name = 'mask_rcnn_resnet101_atrous_coco_2018_01_28/frozen_inference_graph.pb'